In [ ]:
# ============================================================
# Environment Setup (with kernel caching)
# ============================================================
exec(open("/content/drive/MyDrive/MambaCompression/setup_colab.py").read())
!pip install compressai --quiet 2>/dev/null

In [ ]:
# ============================================================
# Cell 2: Full RP-MPQ Analysis (INT8 + Quantization)
# ============================================================
%cd /content/drive/MyDrive/MambaCompression/MambaIC

!python train_ae.py \
  --checkpoint "saved_models/mamba_transnet_L2_dim512_baseline/best.pth" \
  --encoder mamba \
  --decoder transnet \
  --encoded_dim 512 \
  --train_path data/DATA_Htrainout.mat \
  --test_path data/DATA_Htestout.mat \
  --epochs 0 \
  --learning-rate 1e-3 \
  --decoder_layers 2 \
  --encoder_layers 2 \
  --batch-size 200 \
  --num-workers 4 \
  --quant_type asym \
  --pq INT8 \
  --aq 8 \
  --act_quant 16 \
  --analyze_all

In [ ]:
# ============================================================
# Cell 3: FP32 Baseline Inference (No Quantization)
# Expected NMSE: -15.34 dB (outdoor, mamba+transnet, dim=512)
# ============================================================
%cd /content/drive/MyDrive/MambaCompression/MambaIC

!python train_ae.py \
  --checkpoint "saved_models/mamba_transnet_L2_dim512_baseline/best.pth" \
  --encoder mamba \
  --decoder transnet \
  --encoded_dim 512 \
  --train_path data/DATA_Htrainout.mat \
  --test_path data/DATA_Htestout.mat \
  --epochs 0 \
  --decoder_layers 2 \
  --encoder_layers 2 \
  --batch-size 200 \
  --test-batch-size 200 \
  --num-workers 4 \
  --pq FP32 \
  --aq 0 \
  --act_quant 32

In [ ]:
# ============================================================
# Cell 4: Mamba-Transformer AE CR=1/4 Uniform Quantization Sweep
# Weights: INT16 / INT8 / INT4 / INT2,  Activations: INT16 (fixed)
# Latent: 8-bit (--aq 8, CsiNet/CLNet과 동일 조건)
# ============================================================
%cd /content/drive/MyDrive/MambaCompression/MambaIC

BASE = (
    "python train_ae.py"
    " --checkpoint saved_models/mamba_transnet_L2_dim512_baseline/best.pth"
    " --encoder mamba --decoder transnet --encoded_dim 512"
    " --train_path data/DATA_Htrainout.mat --test_path data/DATA_Htestout.mat"
    " --epochs 0 --decoder_layers 2 --encoder_layers 2"
    " --batch-size 200 --test-batch-size 200 --num-workers 2"
    " --aq 8 --act_quant 16"
)

for prec in ["INT16", "INT8", "INT4", "INT2"]:
    print(f"\n{'#'*60}")
    print(f"# Running: W={prec}  A=INT16  Latent=8bit")
    print(f"{'#'*60}")
    !{BASE} --pq {prec}

In [ ]:
# ============================================================
# Cell 5: RP-MPQ Offline Policy Search (ILP + KL Refinement)
# Figure 2: ILP vs KL-refined policy evaluation
# Range: 75% ~ 95% BOPs Saving | Step: 0.5% | KL Candidates: 10
# Output: results/csv/mp_policy_lut_mamba_pruned.csv
#         results/csv/fitting_raw_data_mamba.csv
#         results/plots/exp1_pareto_accuracy_mamba.png
#         ../figures/kl_vs_ilp.pdf (Figure 2)
# ============================================================
%cd /content/drive/MyDrive/MambaCompression/MambaIC

# Remove existing CSVs to force fresh computation with new range
import os
csv_dir = "results/csv"
for f in ["mp_policy_lut_mamba_pruned.csv", "fitting_raw_data_mamba.csv"]:
    p = os.path.join(csv_dir, f)
    if os.path.exists(p):
        os.remove(p)
        print(f"Removed old: {p}")

!python train_ae.py \
  --checkpoint "saved_models/mamba_transnet_L2_dim512_baseline/best.pth" \
  --encoder mamba \
  --decoder transnet \
  --encoded_dim 512 \
  --train_path data/DATA_Htrainout.mat \
  --test_path data/DATA_Htestout.mat \
  --epochs 0 \
  --decoder_layers 2 \
  --encoder_layers 2 \
  --batch-size 200 \
  --test-batch-size 200 \
  --num-workers 2 \
  --pq INT8 \
  --aq 8 \
  --act_quant 16 \
  --analyze_all